In [8]:
import torch
import torch.nn as nn
import numpy as np
import torchvision

import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

import pandas as pd
import json
from PIL import Image

In [9]:
jsonfile="E:\\Sem2\\DL\\Assignments\\A-1\\train_data\\train_annotations.json"

In [10]:
json_data = pd.read_json(jsonfile, lines = True)
# print(json_data)

In [11]:
# for i in json_data:
#     print(i)

In [18]:
annotations=[]
categories=[]
images=[]
info=[]
for i in json_data:
    if(i=='annotations'):
        print('>>>>>',i)
        for j in json_data[i]:
            for k in j:
                temp=[]
                if('image_id' in k.keys()):
                    temp.append(k['image_id'])
                else:
                    temp.append('iiiii')
                    
                if('category_id' in k.keys()):
#                     temp.append(k['category_id'])
                    
                    if(k['category_id']==1):
                        temp.append(0)
                    if(k['category_id']==99):
                        temp.append(1)
                    if(k['category_id']==3):
                        temp.append(2)
                    if(k['category_id']==5):
                        temp.append(3)
                    if(k['category_id']==6):
                        temp.append(4)
                    if(k['category_id']==7):
                        temp.append(5)
                    if(k['category_id']==8):
                        temp.append(6)
                    if(k['category_id']==9):
                        temp.append(7)
                    if(k['category_id']==10):
                        temp.append(8)
                    if(k['category_id']==11):
                        temp.append(9)
                    if(k['category_id']==16):
                        temp.append(10)
                    if(k['category_id']==21):
                        temp.append(11)
                    if(k['category_id']==30):
                        temp.append(12)
                    if(k['category_id']==33):
                        temp.append(13)
                    if(k['category_id']==34):
                        temp.append(14)
                    if(k['category_id']==51):
                        temp.append(15)
                
                img_name='E:/Sem2/DL/Assignments/A-1/train_data/train/'+k['image_id']+'.jpg'
                op=Image.open(img_name)
#                 print(op.size)
                if('bbox' in k.keys()):
                    width=op.size[0]
                    height=op.size[1]
                    x_c=k['bbox'][0]+(k['bbox'][2]/2)
                    y_c=k['bbox'][1]+(k['bbox'][3]/2)
                    temp.append(x_c/width)
                    temp.append(y_c/height)
                    temp.append((k['bbox'][2])/width)
                    temp.append((k['bbox'][3])/height)
                else:
                    temp.append(0)
                    temp.append(0)
                    temp.append(1)
                    temp.append(1)
                    
                if('id' in k.keys()):
                    temp.append(k['id'])
                else:
                    temp.append('ddddd')
                    
                annotations.append(temp)
                
    if(i=='categories'):
        print('-->',i)
        for j in json_data[i]:
            for k in j:
                temp=[]
                temp.append(k['id'])
                temp.append(k['name'])

                categories.append(temp)
                
    if(i=='images'):
        print('==>',i)
        for j in json_data[i]:
            for k in j:
                temp=[]
                temp.append(k['file_name'])
                temp.append(k['id'])
                temp.append(k['width'])
                temp.append(k['height'])

                images.append(temp)
    
    if(i=='info'):
        print('////',i)



>>>>> annotations
--> categories
==> images
//// info


In [19]:
for i in annotations:
    print(i)
    break



['5a197af2-23d2-11e8-a6a3-ec086b02610b', 8, 0.21875, 0.5954484605087014, 0.12749999999999997, 0.19020080321285132, '19688']


In [20]:
for i in categories:
    print(i)

[6, 'bobcat']
[1, 'opossum']
[30, 'empty']
[9, 'coyote']
[3, 'raccoon']
[11, 'bird']
[8, 'dog']
[16, 'cat']
[5, 'squirrel']
[10, 'rabbit']
[7, 'skunk']
[99, 'rodent']
[21, 'badger']
[34, 'deer']
[33, 'car']
[51, 'fox']


In [21]:
for i in images:
    print(i)
    break
    


['5a2176e7-23d2-11e8-a6a3-ec086b02610b.jpg', '5a2176e7-23d2-11e8-a6a3-ec086b02610b', 2048, 1494]


In [26]:
import pandas as pd

## convert your array into a dataframe
df = pd.DataFrame(annotations)
annot_filepath = 'annotations_modified.xlsx'
df.to_excel(annot_filepath,index=False,header=['image_id','category_id','bbox_0','bbox_1','bbox_2','bbox_3','id'])

df = pd.DataFrame(categories)
categories_filepath='categories_modified.xlsx'
df.to_excel(categories_filepath,index=False, header=['id','name'])

df = pd.DataFrame(images)
images_filepath='images_modified.xlsx'
df.to_excel(images_filepath,index=False,header=['file_name','id','width','height'])



In [27]:
catgs=pd.read_excel('C:\\Users\\Subhani\\Documents\\Jupyter DL Assignments\\Assignment_1\\Mine\\categories_modified.xlsx')

catg_id=list(catgs.values[:,0])
catg_name=list(catgs.values[:,1])

print(catg_id)
print(catg_name)

[6, 1, 30, 9, 3, 11, 8, 16, 5, 10, 7, 99, 21, 34, 33, 51]
['bobcat', 'opossum', 'empty', 'coyote', 'raccoon', 'bird', 'dog', 'cat', 'squirrel', 'rabbit', 'skunk', 'rodent', 'badger', 'deer', 'car', 'fox']


In [28]:
annotations=pd.read_excel('C:\\Users\\Subhani\\Documents\\Jupyter DL Assignments\\Assignment_1\\Mine\\annotations_modified.xlsx')

ann_img_id=annotations.values[:,0]
ann_catg_id=list(annotations.values[:,1])

bbx = annotations.values[:,2:6]

ann_bbox_0=list(bbx[:,0])
ann_bbox_1=list(bbx[:,1])
ann_bbox_2=list(bbx[:,2])
ann_bbox_3=list(bbx[:,3])

for i in range(0,len(ann_img_id)):
#     print(ann_catg_id[i],' ',ann_bbox_0[i],' ',ann_bbox_1[i],' ',ann_bbox_2[i],' ',ann_bbox_3[i])
    file_name='train_annotation_modified\\'+ann_img_id[i]+'.txt'
    if(os.path.exists(file_name)):
        append_write='a'
    else:
        append_write='w'
    
    f = open(file_name,append_write)
    a=str(str(ann_catg_id[i])+' '+str(ann_bbox_0[i])+' '+str(ann_bbox_1[i])+' '+str(ann_bbox_2[i])+' '+str(ann_bbox_3[i])+'\n')
    f.write(a)
    f.close()
#     break
print('------------')

# for i in ann_catg_id:
#     print(i)
#     break
# print('------------')

# for i in ann_bbox_0:
#     print(i)
#     break
# print('------------')

# for i in ann_bbox_1:
#     print(i)
#     break
# print('------------')

# for i in ann_bbox_2:
#     print(i)
#     break
# print('------------')

# for i in ann_bbox_3:
#     print(i)
#     break

------------


In [29]:
images=pd.read_excel('C:\\Users\\Subhani\\Documents\\Jupyter DL Assignments\\Assignment_1\\Mine\\images_modified.xlsx')
img_fname=list(images.values[:,0])
img_id=list(images.values[:,1])

for i in range(0,len(img_fname)):
#     print(img_fname[i])
    train_img=open('animals_modified.txt','a')
    train_img.write('/images/train2014/'+img_fname[i]+'\n')
    train_img.close()
#     break
print('------------')

# for i in img_id:
#     print(i)
#     break


------------


In [34]:
# f = open('C:/Users/Subhani/Desktop/trainvalno5k.txt', 'r')
# print(f)
# x = f.readlines()
# f.close()

In [35]:
# # len(x)


# x=set(x)

# len(x)